# 熟悉python web开发的api

In [ ]:
# 安装flask ，web服务器。
# 指令 conda install flask

In [ ]:
from flask import Flask

# 创建一个web服务器
app = Flask(__name__)

@app.route("/bbb")
def haha():
    return "haha"

@app.route("/aaa")
def haha2():
    return "<h1>静夜思</h1>  <h2 style='color:blue'>床前明月光</h2>"

if __name__ == '__main__':
    app.run(port=80)

## 安装websocket

In [ ]:
## 安装socketio支持的库
## conda install -c conda-forge python-socketio
## conda install -c conda-forge eventlet

In [ ]:
import eventlet
import socketio
from flask import Flask


sio = socketio.Server() ## 创建一个socketio的服务器
app = Flask(__name__)

@sio.on('connect')
def connect(sid,env):
    print("websocket建立起来了")
    sio.emit("steer",data={
        'steering_angle':'0',
        'throttle':'1'
    })

if __name__=='__main__':
    # 把web服务器和socketio绑定在一起
    app = socketio.Middleware(sio,app)
    # 监听socket的事件
    eventlet.wsgi.server(eventlet.listen(('',4567)),app)

In [ ]:
import eventlet
import socketio
from flask import Flask
from io import BytesIO
from PIL import Image
import cv2
import base64
import numpy as np
#导包 引入tensorflow的api
import tensorflow as tf


model = tf.keras.models.load_model('001.h5')
sio = socketio.Server() ## 创建一个socketio的服务器
app = Flask(__name__)

speed_limit = 10

@sio.on('connect')
def connect(sid,env):
    print("websocket建立起来了")
    sio.emit("steer",data={
        'steering_angle':'0',
        'throttle':'1'
    })
    
    
@sio.on('telemetry')
def telemetry(sid,data):
    image = Image.open(BytesIO(base64.b64decode(data['image'])))
    image = np.asarray(image)
#    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    cv2.imshow("camera",image)
    cv2.waitKey(10)
    
    ## 基于画面，判断方向盘的角度
    image = image[60:135,:,:]
    image = cv2.GaussianBlur(image,(3,3),1)
    image = cv2.resize(image,(200,66))
    image = cv2.cvtColor(image,cv2.COLOR_RGB2YUV)
    
    image  = image/255.0
    angle = model.predict(image.reshape(1,66,200,3))[0][0]
    
    
    ## 当前的车速和油门
    speed = float(data['speed'])
    throttle = 1.0 - speed/ speed_limit
    
    ## 发送方向盘角度到模拟器
    sio.emit("steer",data={
        'steering_angle':angle.__str__(),
        'throttle': throttle.__str__()
    })
    

if __name__=='__main__':
    # 把web服务器和socketio绑定在一起
    app = socketio.Middleware(sio,app)
    # 监听socket的事件
    eventlet.wsgi.server(eventlet.listen(('',4567)),app)

(11028) wsgi starting up on http://0.0.0.0:4567
(11028) accepted ('127.0.0.1', 53060)


websocket建立起来了


## 带方向盘的演示效果

In [ ]:
import eventlet
import socketio
from flask import Flask
from io import BytesIO
from PIL import Image
import cv2
import base64
import numpy as np
#导包 引入tensorflow的api
import tensorflow as tf


model = tf.keras.models.load_model('001.h5')
sio = socketio.Server() ## 创建一个socketio的服务器
app = Flask(__name__)

speed_limit = 10

@sio.on('connect')
def connect(sid,env):
    print("websocket建立起来了")
    sio.emit("steer",data={
        'steering_angle':'0',
        'throttle':'1'
    })
    
    
@sio.on('telemetry')
def telemetry(sid,data):
    image = Image.open(BytesIO(base64.b64decode(data['image'])))
    image = np.asarray(image)
#    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    cv2.imshow("camera",image)
    cv2.waitKey(10)
    img = cv2.imread('car.jpg')
    rows,cols,_ = img.shape

    
    ## 基于画面，判断方向盘的角度
    image = image[60:135,:,:]
    image = cv2.GaussianBlur(image,(3,3),1)
    image = cv2.resize(image,(200,66))
    image = cv2.cvtColor(image,cv2.COLOR_RGB2YUV)
    
    image  = image/255.0
    angle = model.predict(image.reshape(1,66,200,3))[0][0]
     #按照中心点得到一个旋转矩阵
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-angle*180/3.14,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("new",dst)   
    
    ## 当前的车速和油门
    speed = float(data['speed'])
    throttle = 1.0 - speed/ speed_limit
    
    ## 发送方向盘角度到模拟器
    sio.emit("steer",data={
        'steering_angle':angle.__str__(),
        'throttle': throttle.__str__()
    })
    

if __name__=='__main__':
    # 把web服务器和socketio绑定在一起
    app = socketio.Middleware(sio,app)
    # 监听socket的事件
    eventlet.wsgi.server(eventlet.listen(('',4567)),app)

(12476) wsgi starting up on http://0.0.0.0:4567
(12476) accepted ('127.0.0.1', 51858)


websocket建立起来了


127.0.0.1 - - [21/Oct/2020 16:38:26] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 126.510103
